## qflib Library Examples

In [1]:
import qflib as qf
import numpy as np
import os

print("qflib version: {0}".format(qf.version()))
print("pid: {0}".format(os.getpid()))

qflib version: 0.1.0-debug
pid: 50084


In [2]:
name = "World"
print(qf.sayHello(name))

Hello World!


In [3]:
x = [1, 2, 3]
y = [4, 5]
op = qf.outerProd(x, y)
print(f'x: {x}\ny: {y}')
print(f'outerProd:\n{op}')

x: [1, 2, 3]
y: [4, 5]
outerProd:
[[ 4.  5.]
 [ 8. 10.]
 [12. 15.]]


In [5]:
# test exception
qf.polyProd([],[0])


RuntimeError: Input vectors may not be empty

In [21]:
# test trailing zeros
qf.polyProd([2,0,0], [1,0])

polydeg size: 1


[2.0]

In [7]:
# test case
qf.polyProd([0,2],[1,0,3])

polydeg size: 4


[0.0, 2.0, 0.0, 6.0]

In [22]:
# test pascals triangle
l = [np.array([1.]), np.array([1., 1.])]
for i in range(6):
    l.append(qf.polyProd(l[1], l[i+1]))
l

polydeg size: 3
polydeg size: 4
polydeg size: 5
polydeg size: 6
polydeg size: 7
polydeg size: 8


[array([1.]),
 array([1., 1.]),
 [1.0, 2.0, 1.0],
 [1.0, 3.0, 3.0, 1.0],
 [1.0, 4.0, 6.0, 4.0, 1.0],
 [1.0, 5.0, 10.0, 10.0, 5.0, 1.0],
 [1.0, 6.0, 15.0, 20.0, 15.0, 6.0, 1.0],
 [1.0, 7.0, 21.0, 35.0, 35.0, 21.0, 7.0, 1.0]]